In [1]:
##### %matplotlib inline
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"#不加这行 在跑d2l代码时内核可能崩溃
import torch
import torchvision
from torch.utils import data
from torchvision import transforms
from d2l import torch as d2l
from IPython import display

trans = transforms.ToTensor()
mnist_train = torchvision.datasets.FashionMNIST(root="./data",train=True,transform=trans,download=True)
mnist_test = torchvision.datasets.FashionMNIST(root="./data",train=False,transform=trans,download=True)

In [10]:
def my_add(data, *args):
    data = [a+float(b) for a,b in zip(data,args)] # zip函数把两个列表第一个位置元素打包、第二个位置元素打包....
    return data
data = [1,2,3]
data = my_add(data,2,3,4)
print(data)

[3.0, 5.0, 7.0]
